Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
beta = 0.0001

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) +
    beta * tf.nn.l2_loss(weights))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
%%time

num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.917896
Training accuracy: 8.1%
Validation accuracy: 10.7%
Loss at step 100: 2.533466
Training accuracy: 71.3%
Validation accuracy: 70.5%
Loss at step 200: 2.079458
Training accuracy: 74.8%
Validation accuracy: 73.1%
Loss at step 300: 1.831346
Training accuracy: 76.4%
Validation accuracy: 74.4%
Loss at step 400: 1.659367
Training accuracy: 77.2%
Validation accuracy: 75.0%
Loss at step 500: 1.529535
Training accuracy: 78.1%
Validation accuracy: 75.4%
Loss at step 600: 1.426360
Training accuracy: 78.9%
Validation accuracy: 75.6%
Loss at step 700: 1.341468
Training accuracy: 79.5%
Validation accuracy: 75.7%
Loss at step 800: 1.269900
Training accuracy: 79.9%
Validation accuracy: 75.8%
Test accuracy: 83.0%
Wall time: 8.53 s


In [7]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  relu_size = image_size * image_size * 2

  weights_into_relu = tf.Variable(tf.truncated_normal([image_size * image_size, relu_size]))
  weights_out_relu  = tf.Variable(tf.truncated_normal([relu_size, num_labels]))
  biases_into_relu = tf.Variable(tf.zeros([relu_size]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights_into_relu) + biases_into_relu), 
                     weights_out_relu) + biases
      

  beta = tf.Variable(tf.truncated_normal([1]))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) +
    0.00001*tf.nn.l2_loss(weights_out_relu))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_into_relu) + biases_into_relu), 
                     weights_out_relu) + biases)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_into_relu) + biases_into_relu), 
                     weights_out_relu) + biases)

In [8]:
%%time

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 457.678925
Minibatch accuracy: 9.4%
Validation accuracy: 33.5%
Minibatch loss at step 500: 31.749363
Minibatch accuracy: 82.0%
Validation accuracy: 80.6%
Minibatch loss at step 1000: 17.629135
Minibatch accuracy: 78.9%
Validation accuracy: 82.3%
Minibatch loss at step 1500: 7.614439
Minibatch accuracy: 89.1%
Validation accuracy: 82.3%
Minibatch loss at step 2000: 4.304259
Minibatch accuracy: 87.5%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 4.060155
Minibatch accuracy: 82.8%
Validation accuracy: 83.2%
Minibatch loss at step 3000: 2.624319
Minibatch accuracy: 82.8%
Validation accuracy: 83.3%
Test accuracy: 90.7%
Wall time: 12.9 s


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 100
beta = 0

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) +
    beta * tf.nn.l2_loss(weights))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [10]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 19.175400
Training accuracy: 9.0%
Validation accuracy: 15.7%
Loss at step 100: 0.020012
Training accuracy: 100.0%
Validation accuracy: 44.1%
Loss at step 200: 0.007957
Training accuracy: 100.0%
Validation accuracy: 45.7%
Loss at step 300: 0.005051
Training accuracy: 100.0%
Validation accuracy: 46.3%
Loss at step 400: 0.003723
Training accuracy: 100.0%
Validation accuracy: 46.7%
Loss at step 500: 0.002958
Training accuracy: 100.0%
Validation accuracy: 47.0%
Loss at step 600: 0.002459
Training accuracy: 100.0%
Validation accuracy: 47.4%
Loss at step 700: 0.002107
Training accuracy: 100.0%
Validation accuracy: 47.6%
Loss at step 800: 0.001845
Training accuracy: 100.0%
Validation accuracy: 47.7%
Test accuracy: 53.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [11]:
batch_size = 128
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  relu_size = image_size * image_size * 2

  weights_into_relu = tf.Variable(tf.truncated_normal([image_size * image_size, relu_size]))
  weights_out_relu  = tf.Variable(tf.truncated_normal([relu_size, num_labels]))
  biases_into_relu = tf.Variable(tf.zeros([relu_size]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
#   logits = tf.matmul(tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_into_relu) + biases_into_relu),
#                                    0.8), 
#                      weights_out_relu) + biases
  logits_train = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights_into_relu) + biases_into_relu),
                     tf.nn.dropout(weights_out_relu, 0.5)) + biases
  logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights_into_relu) + biases_into_relu),
                     weights_out_relu) + biases
      

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_train) +
    beta*tf.nn.l2_loss(weights_out_relu))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_into_relu) + biases_into_relu), 
                     weights_out_relu) + biases)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_into_relu) + biases_into_relu), 
                     weights_out_relu) + biases)

In [12]:
len(train_labels)

200000

In [13]:
%%time
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 680.739746
Minibatch accuracy: 4.7%
Validation accuracy: 25.0%
Minibatch loss at step 500: 85.270905
Minibatch accuracy: 78.9%
Validation accuracy: 78.5%
Minibatch loss at step 1000: 43.461433
Minibatch accuracy: 77.3%
Validation accuracy: 79.1%
Minibatch loss at step 1500: 23.967766
Minibatch accuracy: 82.8%
Validation accuracy: 78.5%
Minibatch loss at step 2000: 10.324895
Minibatch accuracy: 84.4%
Validation accuracy: 78.2%
Minibatch loss at step 2500: 11.540102
Minibatch accuracy: 77.3%
Validation accuracy: 79.2%
Minibatch loss at step 3000: 7.931354
Minibatch accuracy: 78.1%
Validation accuracy: 79.5%
Test accuracy: 86.5%
Wall time: 18 s


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [27]:
# some article says batch+dropout may avoid L2-loss
batch_size = 1024
beta = [0.2, 0.05, 0.01, 0.002, 0.0004]
relu_size = [image_size * image_size, 1024, 128, num_labels]
layers_count = len(relu_size) - 1

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
    
  weights_relu = []
  biases_relu = []
  
  for i in range(layers_count):
        weights_relu.append(tf.Variable(tf.truncated_normal([relu_size[i],relu_size[i+1]], 
                                                            mean=0.0, # seem has no effect on the result...
                                                            stddev=np.sqrt(2.0 / relu_size[i]))))
        #  stddev=np.sqrt(2.0 / relu_size[i]) is important, or the loss will be Nan because of too large a value
        biases_relu.append(tf.Variable(tf.zeros([relu_size[i+1]])))

  
  # Training computation.
  logits_t = tf_train_dataset
  for i in range(layers_count-1):
    logits_t = tf.nn.relu(tf.nn.dropout(tf.matmul(logits_t, weights_relu[i]) + biases_relu[i], 0.5))
  logits_t = tf.matmul(logits_t, weights_relu[layers_count-1]) + biases_relu[layers_count-1]
    
  logits = tf_train_dataset
  for i in range(layers_count-1):
    logits = tf.nn.relu(tf.matmul(logits, weights_relu[i]) + biases_relu[i])
  logits = tf.matmul(logits, weights_relu[layers_count-1]) + biases_relu[layers_count-1]

  loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_t)
#   for i in range(layers_count-1):
#     lost = tf.add(loss, beta[i] * tf.nn.l2_loss(weights_relu[i])) # must add L2-loss layer by layer with different beta
     
  loss = tf.reduce_mean(loss)
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  decay_rate = 0.95
  decay_steps = 200
  learning_rate = tf.train.exponential_decay(decay_rate, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_prediction = tf_valid_dataset
  for i in range(layers_count-1):
    valid_prediction = tf.nn.relu(tf.matmul(valid_prediction, weights_relu[i]) + biases_relu[i])
  valid_prediction = tf.matmul(valid_prediction, weights_relu[layers_count-1]) + biases_relu[layers_count-1]
  valid_prediction = tf.nn.softmax(valid_prediction)
  

  test_prediction = tf_test_dataset
  for i in range(layers_count-1):
    test_prediction = tf.nn.relu(tf.matmul(test_prediction, weights_relu[i]) + biases_relu[i])
  test_prediction = tf.matmul(test_prediction, weights_relu[layers_count-1]) + biases_relu[layers_count-1]
  test_prediction = tf.nn.softmax(test_prediction)

In [28]:
%%time
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.686494
Minibatch accuracy: 9.7%
Validation accuracy: 32.6%
Minibatch loss at step 500: 0.497840
Minibatch accuracy: 86.4%
Validation accuracy: 86.9%
Minibatch loss at step 1000: 0.436005
Minibatch accuracy: 89.2%
Validation accuracy: 87.9%
Minibatch loss at step 1500: 0.406748
Minibatch accuracy: 89.6%
Validation accuracy: 88.5%
Minibatch loss at step 2000: 0.426057
Minibatch accuracy: 89.1%
Validation accuracy: 89.0%
Minibatch loss at step 2500: 0.355281
Minibatch accuracy: 90.6%
Validation accuracy: 89.5%
Minibatch loss at step 3000: 0.333611
Minibatch accuracy: 92.2%
Validation accuracy: 89.6%
Test accuracy: 95.4%
Wall time: 14.4 s


---
Results
---------

3001 steps

```python
batch_size = 1024
beta = [0.2, 0.05, 0.01, 0.002, 0.0004]
relu_size = [image_size * image_size, 1024, 512, 256, 128, num_labels]
layers_count = len(relu_size) - 1
```

- Without L2-loss: 95.5%
- With L2-loss 95.5%

```python
batch_size = 1024
beta = [0.2, 0.05, 0.01, 0.002, 0.0004]
relu_size = [image_size * image_size, 1024, 512, 64, num_labels]
layers_count = len(relu_size) - 1
```
- Without L2-loss: 95.4%
